# 一、chorma使用

In [2]:
# ! pip install "chromadb>=0.4.5"
! pip install chromadb --upgrade

ERROR: Could not install packages due to an OSError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'E:\\program\\anaconda\\envs\\llamaIndex\\Scripts\\chroma.exe' -> 'E:\\program\\anaconda\\envs\\llamaIndex\\Scripts\\chroma.exe.deleteme'
Consider using the `--user` option or check the permissions.




  Using cached chromadb-1.0.6-cp39-abi3-win_amd64.whl.metadata (7.0 kB)
Using cached chromadb-1.0.6-cp39-abi3-win_amd64.whl (18.2 MB)


In [ ]:
#! chroma run --path ../data/db --port 8001

^C


In [ ]:
import chromadb
chroma_client = chromadb.HttpClient(host='localhost',port=8001)

In [7]:
# 指定embedding模型
from chromadb.utils import embedding_functions
model_path = "../model/nlp_gte_sentence"
em_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = model_path)

No sentence-transformers model found with name ../model/nlp_gte_sentence. Creating a new one with mean pooling.


In [8]:
# 创建向量集合
collection = chroma_client.create_collection(name='rag_db',
                                             embedding_function=em_fn,
                                             metadata={"hnsw:space":"cosine"})

In [11]:
documents=["在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。",
"虽然单独使用特定技术如倒排文件（IVF）、乘积量化（PQ）或分层导航小世界（HNSW）通常能够带来满意的结果",
"GraphRAG本质上就是RAG，只不过与一般RAG相比，其检索路径上多了一个知识图谱"]

In [13]:
# 插入文档
collection.add(documents=documents,
               ids=["id1","id2","id3"],# 插入文档的主键，实际业务中可以使用uuid
               metadatas=[{"chapter":3,"verse":16},
                          {"chapter":4,"verse":5},
                          {"chapter":12,"verse":5}]# 元数据通过metadata设定
               )

In [14]:
collection.count()

3

In [15]:
collection.peek(limit=1)

{'ids': ['id1'],
 'embeddings': array([[ 0.3879384 , -0.14354113,  0.41287565, ..., -0.47161332,
          0.57718426, -0.39078963]]),
 'metadatas': [{'verse': 16, 'chapter': 3}],
 'documents': ['在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。'],
 'data': None,
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings']}

## 1.1检索

In [ ]:
# 获取某一个db
get_collection = chroma_client.get_collection(name='rag_db',
                                              embedding_function=em_fn)

In [17]:
id_result= get_collection.get(ids=['id2'],
                                include=["documents","embeddings", "metadatas"])

In [18]:
id_result['documents']

['虽然单独使用特定技术如倒排文件（IVF）、乘积量化（PQ）或分层导航小世界（HNSW）通常能够带来满意的结果']

In [19]:
id_result['metadatas']

[{'verse': 5, 'chapter': 4}]

In [20]:
import numpy as np
np.array(id_result['embeddings']).shape

(1, 1024)

In [21]:
query ="索引技术有哪些？"

In [ ]:
get_collection.query(query_texts=query,
                    n_results=2,
                    include=["documents",'metadatas'])

{'ids': [['id1', 'id3']],
 'distances': None,
 'embeddings': None,
 'metadatas': [[{'verse': 16, 'chapter': 3}, {'verse': 5, 'chapter': 12}]],
 'documents': [['在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。',
   'GraphRAG本质上就是RAG，只不过与一般RAG相比，其检索路径上多了一个知识图谱']],
 'uris': None,
 'data': None,
 'included': ['documents', 'metadatas']}

## 1.2不同的检索条件，过滤数据

In [23]:
# 指定具体的verse
get_collection.query(query_texts=query,
                    n_results=2,
                    include=["documents",'metadatas'],
                    where={"verse":5})

{'ids': [['id3', 'id2']],
 'distances': None,
 'embeddings': None,
 'metadatas': [[{'chapter': 12, 'verse': 5}, {'chapter': 4, 'verse': 5}]],
 'documents': [['GraphRAG本质上就是RAG，只不过与一般RAG相比，其检索路径上多了一个知识图谱',
   '虽然单独使用特定技术如倒排文件（IVF）、乘积量化（PQ）或分层导航小世界（HNSW）通常能够带来满意的结果']],
 'uris': None,
 'data': None,
 'included': ['documents', 'metadatas']}

In [ ]:
get_collection.query(query_texts=["索引技术有哪些？"],
                    n_results=2,
                    where={"chapter": {"$lt": 10}},)

{'ids': [['id1', 'id2']],
 'distances': [[0.45845127, 0.6852661]],
 'embeddings': None,
 'metadatas': [[{'verse': 16, 'chapter': 3}, {'verse': 5, 'chapter': 4}]],
 'documents': [['在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。',
   '虽然单独使用特定技术如倒排文件（IVF）、乘积量化（PQ）或分层导航小世界（HNSW）通常能够带来满意的结果']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

## 1.3 文档和数据的混合检索

In [ ]:
get_collection.query(query_texts=["索引技术有哪些？"],
                    n_results=2,
                    where_document={"$contains":"索引"}) # contains关键字表明，文档中含有索引这两个字的

{'ids': [['id1']],
 'distances': [[0.45845127]],
 'embeddings': None,
 'metadatas': [[{'chapter': 3, 'verse': 16}]],
 'documents': [['在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

# 二、milvus的使用

2.1 安装

In [27]:
! pip install pymilvus

   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   -------------- ------------------------- 1.6/4.4 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 4.4/4.4 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.71.0
    Uninstalling grpcio-1.71.0:
      Successfully uninstalled grpcio-1.71.0


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.128 requires chromadb<0.6.0,>=0.5.10, but you have chromadb 1.0.6 which is incompatible.
grpcio-tools 1.71.0 requires grpcio>=1.71.0, but you have grpcio 1.67.1 which is incompatible.


In [1]:
import numpy as np
from pymilvus import(
connections,
utility,
FieldSchema, CollectionSchema, DataType,
Collection, 
)


In [2]:
connections.connect(host='127.0.0.1',port="19530")

In [3]:
fileds =[
    FieldSchema(name="pk", dtype=DataType.VARCHAR,
    is_primary=True, auto_id=False, max_length=100),
    FieldSchema(name="documents",dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="embeddings",dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="verse",dtype=DataType.INT64), # 元数据信息
]

In [4]:
rag_db = Collection("rag_db",
    CollectionSchema(fileds),
    consistency_level="Strong")

In [5]:
documents=["在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。",
"虽然单独使用特定技术如倒排文件（IVF）、乘积量化（PQ）或分层导航小世界（HNSW）通常能够带来满意的结果",
"GraphRAG本质上就是RAG，只不过与一般RAG相比，其检索路径上多了一个知识图谱"]

In [ ]:
# 需要手动的将文档转换成向量，然后插入到向量集合中，需要emb模型
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
model_path = "../model/nlp_gte_sentence"
model = HuggingFaceEmbeddings(model_name=model_path,
                            model_kwargs={'device':"cpu"})
embeddings= model.embed_documents(documents)

C:\Users\Yun\AppData\Local\Temp\ipykernel_60768\2549077778.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceEmbeddings(model_name=model_path,
No sentence-transformers model found with name ../model/nlp_gte_sentence. Creating a new one with mean pooling.


In [7]:
entities = [ 
[str(i) for i in range(len(documents))],  # 主键pk
documents, # 文档
np.array(embeddings), # embeddings是手动转化好的 
[16,5,5], # verse元数据信息
]

In [12]:
# 插入数据
insert_result = rag_db.insert(entities)
print(insert_result)

(insert count: 3, delete count: 0, upsert count: 0, timestamp: 457570629784698881, success count: 3, err count: 0


In [13]:
rag_db.flush

<bound method Collection.flush of <Collection>:
-------------
<name>: rag_db
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'documents', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'verse', 'description': '', 'type': <DataType.INT64: 5>}], 'enable_dynamic_field': False}
>

In [14]:
rag_db.num_entities

0

## 2.1 创建索引

In [18]:
index = {
"index_type": "IVF_FLAT",# 倒排索引
"metric_type":"L2" , #　欧式距离
"params":{"nlist": 128}, # 聚类的数量
}

In [19]:
rag_db.create_index("embeddings",index)

Status(code=0, message=)

## 2.2 检索
需要确定检索的是哪个向量集合

In [26]:
get_collection = Collection("rag_db")
get_collection.load()

In [60]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np

# 指定模型路径
model_path = "../model/nlp_gte_sentence"

# 初始化模型
model = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs={'device': 'cpu'}
)

# 同步生成嵌入
query="索引技术有哪些？"
query_emb = model.embed_documents([query])  # 输入必须是列表

# 输出形状
print(np.array(query_emb).shape)

No sentence-transformers model found with name ../model/nlp_gte_sentence. Creating a new one with mean pooling.


(1, 1024)


In [61]:
result = get_collection.search (query_emb,
                                "embeddings",
                                param={"metric_type": "L2"},
                                limit=2,
                                output_fields=["documents","verse"])

In [59]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np

# 指定模型路径（确保路径正确）
model_path = "../model/nlp_gte_sentence"

# 初始化模型
model = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs={'device': 'cpu'}
)

# 同步生成嵌入
query = "你的查询文本"
query_emb = model.embed_documents([query])  # 输入必须是列表

# 输出形状（示例：(1, 768)）
print(np.array(query_emb).shape)

No sentence-transformers model found with name ../model/nlp_gte_sentence. Creating a new one with mean pooling.


(1, 1024)


In [62]:
result = get_collection.search(query_emb,
                            "embeddings",
                            param={"metric_type": "L2"},
                            limit=2,
                            output_fields=["documents","verse"])

In [63]:
result

data: [[{'id': '0', 'distance': 421.4751281738281, 'entity': {'verse': 16, 'documents': '在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。'}}]]

In [ ]:
# 看一下检索的结果
for hits in result:
    for hit in hits:
        print(f"hit: {hit}, documents field: {hit.entity.get('documents')}")
        

hit: {'id': '0', 'distance': 421.4751281738281, 'entity': {'verse': 16, 'documents': '在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。'}}, documents field: 在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。


## 2.3  按条件混合检索

In [65]:
## 2.3  按条件混合检索
result2 = get_collection.search(query_emb,
                                "embeddings",
                                param={"metric_type": "L2"},
                                expr="verse < 10",
                                limit=1,
                                output_fields=["documents", "verse"])

In [66]:
for hits in result:
    for hit in hits:
        print(f"hit: {hit}, documents field: {hit.entity.get('documents')}")

hit: {'id': '0', 'distance': 421.4751281738281, 'entity': {'verse': 16, 'documents': '在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。'}}, documents field: 在向量搜索领域，我们拥有多种索引方法和向量处理技术，它们使我们能够在召回率、响应时间和内存使用之间做出权衡。


##　2.4 Index索引优化

In [67]:
import numpy as np
from scipy.cluster.vq import kmeans2
query = np.random.normal(size=(128,))
dataset = np.random.normal(size=(1000, 128))

全表扫描

In [68]:
np.argmin(np.linalg.norm(query - dataset, axis=1))

759

倒排索引IVF，kmeans分块，检索最近的块

In [70]:
num_part = 100
(centroids, assignments)= kmeans2(dataset, num_part, iter=1000)

e:\program\anaconda\envs\llamaIndex\lib\site-packages\scipy\_lib\_util.py:440: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  return fun(*args, **kwargs)


In [72]:
centroids.shape


(100, 128)

In [73]:
assignments[:10]

array([31, 61, 81, 28, 94, 98, 65,  5, 56,  5])

In [74]:
# 不同的块包含不同的数据
index = [[] for _ in range(num_part)]
for n,k in enumerate(assignments):
    index[k].append(n)

In [75]:
index[1]

[50, 269, 386]

然后在不同的块内做全表扫描

In [77]:
cluster_id = np.argmin(np.linalg.norm(query - centroids, axis=1))

In [78]:
cluster_id

32

In [79]:
np.argmin(np.linalg.norm(query - dataset[index[30]], axis=1))

1

可以取top3做全表扫描

In [80]:
cluster_ids = np.argsort(np.linalg.norm(query - centroids, axis=1))[: 3]

In [81]:
cluster_ids

array([32, 75,  9], dtype=int64)

In [82]:
top3_index= []
for c in cluster_ids:
    top3_index += index[c]
np.argmin(np.linalg.norm(query - dataset[top3_index], axis=1))

92

In [83]:
top3_index[66]

445